###Primo Step: Docling

In [ ]:
!pip install docling

In [2]:
from docling.document_converter import DocumentConverter

source = "/storage/data_4T_b/andreacutuli/PROVA/FAC-SIMILE 1.1.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"



/storage/data_4T_b/andreacutuli/miniconda3/envs/retrievers/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## EcoTech Solutions S.r.l.

per GreenLife S.p.A. Via dell'Innovazione 42, 20121 Milano (MI), Italia

Soluzioni di Intelligenza Artificiale per l'Ottimizzazione del Processo di Riciclo Industriale

Implementazione di Sistemi AI per l'Analisi dei Materiali e l'Efficienza Energetica

## Proposta contrattuale

Versione: 1.3 Validità: 60 giorni

Destinatario: Marco Bianchi m.bianchi@greenlifespa.eu Laura Verdi l.verdi@greenlifespa.eu Andrea Rossi a.rossi@greenlifespa.eu

Redatta da: Dott.ssa Elena Marino elena.marino@ecotech.srl

Revisionata da: Prof. Paolo Ferretti ferretti@polimi.it

Per EcoTech Solutions S.r.l.

Per GreenLife S.p.A.

## 1. Introduzione

## 1.1. Scopo del Documento

Scopo del presente documento è descrivere la proposta contrattuale di EcoTech Solutions S.r.l. (in seguito EcoTech) verso GreenLife S.p.A. (in seguito GreenLife oppure Committente), per la realizzazione del progetto:

## Soluzioni  di  Intelligenza  Artificiale  per  l'Ottimizzazione  del  Processo  di  Ricic

In [3]:
file_name = "result_document.md"  # nome del file
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.document.export_to_markdown())

print(f"File salvato come {file_name}")



File salvato come result_document.md


In [4]:
# This is a long document we can split up.
with open("/storage/data_4T_b/andreacutuli/PROVA/result_document.md") as f:
    fac_simile = f.read()

In [5]:
print(fac_simile[:500])

## EcoTech Solutions S.r.l.

per GreenLife S.p.A. Via dell'Innovazione 42, 20121 Milano (MI), Italia

Soluzioni di Intelligenza Artificiale per l'Ottimizzazione del Processo di Riciclo Industriale

Implementazione di Sistemi AI per l'Analisi dei Materiali e l'Efficienza Energetica

## Proposta contrattuale

Versione: 1.3 Validità: 60 giorni

Destinatario: Marco Bianchi m.bianchi@greenlifespa.eu Laura Verdi l.verdi@greenlifespa.eu Andrea Rossi a.rossi@greenlifespa.eu

Redatta da: Dott.ssa Elena M


###Secondo step: Semantic Chunking

In [6]:
!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas

In [7]:
!pip install -qU faiss-cpu tiktoken

In [8]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

# Usa un modello di embedding gratuito (senza chiave)
embedding_model = HuggingFaceEmbeddings(
    #model_name="sentence-transformers/LaBSE"
    model_name="intfloat/multilingual-e5-large"
)

from sentence_transformers import SentenceTransformer
#model = SentenceTransformer("sentence-transformers/LaBSE")
model=embedding_model


# Inizializza il semantic chunker con quel modello
semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile"
)


/tmp/ipykernel_2159152/3556344562.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [ ]:
'''semantic_chunker = SemanticChunker(
    embeddings=embedding_model,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    min_chunk_size=300,
    max_chunk_size=1000,
    buffer_size=50,
    embedding_batch_size=8
)
semantic_chunks = semantic_chunker.create_documents([fac_simile])'''

In [9]:
semantic_chunks = semantic_chunker.create_documents([fac_simile])

In [10]:
print(f"Numero di chunk creati: {len(semantic_chunks)}")

Numero di chunk creati: 13


In [11]:
for semantic_chunk in semantic_chunks:
  if "zero" in semantic_chunk.page_content:
    print(semantic_chunk.page_content)
    print(len(semantic_chunk.page_content))

###Terzo step: Embedding

In [ ]:
!pip install -U langchain-huggingface

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
# Usa un modello open-source per gli embeddings (gratuito, nessuna chiave API richiesta)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

###Quarto step: Vector stores

In [15]:
!pip install -qU langchain-core

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [17]:
from langchain.vectorstores import FAISS

# Crea il vector store da una lista di Document
vector_store = FAISS.from_documents(semantic_chunks, embedding_model)

In [18]:
# Ora puoi cercare:
query = "costi di gestione"
results = vector_store.similarity_search(query, k=3)

# Stampa i risultati
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---\n{doc.page_content}")


--- Result 1 ---
Offerta Economica

## 4.1. Valore Economico dell'Intervento

Sulla base della soluzione proposta, si stima un valore complessivo di:

## 65.000 € (sessantacinquemila euro)

Tale valorizzazione è stimata considerando la seguente Tabella 2. |   Attività | Costo [€]   |
|------------|-------------|
|          1 | 25.000 €    |
|          2 | 20.000 €    |

3

20.000 €

Tabella 2. Costo per Attività. L'offerta economica complessiva include:

- ● le prestazioni erogate dagli specialisti EcoTech;
- ● coordinamento gestionale del progetto. I valori sopra riportati sono da intendersi IVA esclusa.

--- Result 2 ---
## 4.3. Assistenza e Manutenzione

EcoTech  fornirà  un  servizio  di  assistenza  e  manutenzione  incluso  nel  progetto  per  un periodo di 180 giorni a partire dall'ultimo rilascio del software (D3). Durante questo periodo, il servizio coprirà la risoluzione di bug e la realizzazione di modifiche minori, che verranno concordate tra EcoTech e il Committente. Succ

In [19]:
#results = await vector_store.asimilarity_search("sicurezza")
results = vector_store.similarity_search("quanto è sicuro per la privacy", k=3)


for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---\n{doc.page_content}")


--- Result 1 ---
Condizioni Generali

## 5.1. Clausole di Riservatezza

Nell'esecuzione dell'attività sono considerate riservate soltanto le informazioni chiaramente identificate come tali. Non sono comunque considerate riservate le informazioni:

- ● che siano di pubblico dominio o diventino tali a seguito di azioni od omissioni da parte del Committente,
- ● che siano  state  legittimamente  comunicate  a  EcoTech  da  terze  parti  senza  obbligo  di riservatezza,
- ● che siano state sviluppate da EcoTech indipendentemente dal Committente,
- ● che siano rese pubbliche per legge,
- ● che siano conosciute da EcoTech prima di averle ricevute dal Committente. EcoTech  si  impegna  a  mantenere  il  più  stretto  riserbo  sulle  informazioni  riservate, trattandole con diligenza non inferiore a quella con cui sono trattate le proprie. EcoTech si impegna inoltre a non eseguire alcun trattamento, comunicazione, diffusione o utilizzo, al di fuori delle finalità della presente offerta, dei d

In [20]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("informatica del progetto")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.34752506017684937

page_content='Organizzazione del Progetto

## 3.1. Risorse e Planning

Sulla base della configurazione descritta, la durata prevista del progetto è di circa 6 mesi, a partire dalla Riunione di avvio progetto debitamente descritta nel Paragrafo 3.2. Controllo e Gestione. Le  attività  operative  saranno  gestite  attraverso  una  modalità  di  lavoro  basata  sulla metodologia AGILE con sprint bisettimanali. Sarà prevista una riunione ogni due settimane di circa 1 ora, che si terrà da remoto in un giorno della settimana da concordare tra le parti, per  la  revisione  di  quanto  sviluppato  e  la  pianificazione  delle  attività  da  eseguire  nei successivi sprint. Lo sviluppo verrà realizzato sfruttando l'infrastruttura di calcolo (es. virtual machine) di EcoTech ed il codice verrà storicizzato in un apposito repository (GIT). Il codice e i modelli realizzati verranno poi messi a disposizione del Committente per il deployment sulla propria infrastruttura. S